In [ ]:
# Perform necessary imports
from dotenv import load_dotenv
import os
import requests
import json
from datetime import datetime
import pandas as pd
import time

# Load variables from .env file
load_dotenv()

In [ ]:
# Read environment variables
BASE_URL = os.getenv("BASE_URL")
OAUTH_ENDPOINT = os.getenv("OAUTH_ENDPOINT")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
CLIENT_ID = os.getenv("CLIENT_ID")
REFRESH_TOKEN = os.getenv("REFRESH_TOKEN")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
AUTHORIZATION_CODE = os.getenv("AUTHORIZATION_CODE")

In [ ]:
# Fetch the basic user profile of the user
athlete_profile = f'{BASE_URL}/athlete'
headers = {
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

response = requests.get(athlete_profile, headers=headers)

if response.status_code == 200:
    athelete_basic_data = response.json()
    print(json.dumps(athelete_basic_data, indent=4))
else:
    print("Error: ", response)

In [ ]:
# Automated Retrieval of Tokens - Code flow: if strava_tokens.json file doesn't exist perform OAUTH authorization using the authorization code generated manually (this step is done only once)
# If the file exists, then use that file's access_token field for fetching data further. Also while using the file, we need to ensure that the token is not expired, if expired, re-generate the access token using refresh token
strava_tokens_file = 'strava_tokens.json'

if os.path.exists(strava_tokens_file):

    # Get the tokens from file to connect to Strava
    with open('strava_tokens.json') as json_file:
        strava_tokens = json.load(json_file)

    # If access_token has expired then 
    # use the refresh_token to get the new access_token
    # Make Strava auth API call with current refresh token
    if strava_tokens['expires_at'] < time.time():
        response = requests.post(
                            OAUTH_ENDPOINT,
                            data = {
                                    'client_id': CLIENT_ID,
                                    'client_secret': CLIENT_SECRET,
                                    'grant_type': 'refresh_token',
                                    'refresh_token': strava_tokens['refresh_token']
                                    }
                        )
        new_strava_tokens = response.json()
    
        # Save new tokens to file
        with open('strava_tokens.json', 'w') as outfile:
            json.dump(new_strava_tokens, outfile)
    
        # Use new Strava tokens from now
        strava_tokens = new_strava_tokens

else:
    # Perform OAuth authentication to generate access tokens for reading activities
    params = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'code': AUTHORIZATION_CODE,
        'grant_type': 'authorization_code'
    }

    authentication_response = requests.post(OAUTH_ENDPOINT, params=params)

    if authentication_response.status_code == 200:
        authentication_response_data = authentication_response.json()
        print(json.dumps(authentication_response_data, indent=4))
    else:
        print("Error: ", authentication_response)

    # Save tokens to file
    with open('strava_tokens.json', 'w') as outfile:
        json.dump(authentication_response_data, outfile)

In [ ]:
# Load the strava_tokens json file that was generated from the previous step
with open('strava_tokens.json') as json_file:
    strava_tokens = json.load(json_file)

#Loop through all activities
page = 1
activities_endpoint = f'{BASE_URL}/activities'
access_token = strava_tokens['access_token']

## Create the dataframe ready for the API call to store your activity data
activities = pd.DataFrame(
    columns = [
            "id",
            "name",
            "start_date_local",
            "type",
            "distance",
            "moving_time",
            "elapsed_time",
            "total_elevation_gain",
            "end_latlng",
            "external_id"
    ]
)

while True:
    # get page of activities from Strava
    activities_response = requests.get(f'{activities_endpoint}?access_token={access_token}&per_page=200&page={str(page)}')
    response_json = activities_response.json()
    
    # if no results then exit loop
    if (not response_json):
        break
    
    # otherwise add new data to dataframe
    for x in range(len(response_json)):
        activities.loc[x + (page-1)*200,'id'] = response_json[x]['id']
        activities.loc[x + (page-1)*200,'name'] = response_json[x]['name']
        activities.loc[x + (page-1)*200,'start_date_local'] = response_json[x]['start_date_local']
        activities.loc[x + (page-1)*200,'type'] = response_json[x]['type']
        activities.loc[x + (page-1)*200,'distance'] = response_json[x]['distance']
        activities.loc[x + (page-1)*200,'moving_time'] = response_json[x]['moving_time']
        activities.loc[x + (page-1)*200,'elapsed_time'] = response_json[x]['elapsed_time']
        activities.loc[x + (page-1)*200,'total_elevation_gain'] = response_json[x]['total_elevation_gain']
        activities.loc[x + (page-1)*200,'end_latlng'] = response_json[x]['end_latlng']
        activities.loc[x + (page-1)*200,'external_id'] = response_json[x]['external_id']
    
    # increment page
    page += 1

# write the records to a CSV file
activities.to_csv('strava_activities.csv')